In [17]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_excel('../data/블록-계획데이터(예제)_수정.xlsx')

In [3]:
df

,정반_코드,호선_코드,블록,단위블록_CODE,공종_명칭,stage_코드,선종_코드,취부팀_코드,용접팀_코드,착수계획,...,완료실적,실적공기,H00,H01,H02,W,B,L,H,A
0,M112,TK001,S18S0,06A0,대조중조,S031,TK,XXX,XXX,2022-12-01,...,2022-12-30 00:00:00,19.0,8,312,649,0,11.5,20.0,8.5,230.0
1,M112,CN047,S22P0,11A0,Final조립,S032,CN,XXX,XXX,2022-12-01,...,2022-12-27 00:00:00,15.0,26,1469,764,150,17.5,21.0,8.0,367.5
2,M212,CN047,S22S0,11A0,Final조립,S032,CN,B02,B02,2022-12-01,...,2022-12-23 00:00:00,22.0,0,1469,766,150,17.5,21.0,8.0,367.5
3,M024,CN011,E22P0,11A0,곡중조,S021,CN,B04,B04,2022-12-01,...,2022-12-09 00:00:00,7.0,17,475,914,0,13.0,18.0,4.5,234.0
4,M211,BC001,F51P0,05A0,Final조립,S032,BC,B02,B02,2022-12-01,...,2022-12-13 00:00:00,15.0,0,623,386,126,17.5,23.0,6.0,402.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12127,M222,TK005,S18P0,03A0,곡중조,S221,TK,B09,B09,2025-04-29,...,,NaN,0,315,0,0,6.0,20.0,8.5,120.0
12128,M222,TK005,S18P0,03B0,곡중조,S221,TK,B09,B09,2025-04-29,...,,NaN,0,641,0,0,7.5,20.0,8.5,150.0
12129,M221,TK005,S18S0,03A0,곡중조,S221,TK,B09,B09,2025-04-29,...,,NaN,0,315,0,0,6.0,20.0,8.5,120.0
12130,M010,TK005,S20P0,11A1,판계,XXX,TK,A11,A11,2025-04-29,...,,NaN,4,35,35,0,17.0,17.5,8.0,297.5


In [5]:
def generating_df_team_name(work, team_name, factory=None):
    df = pd.read_excel('../data/블록-계획데이터(예제)_수정.xlsx')

    df['착수실적'] = pd.to_datetime(df['착수실적'], errors='coerce')
    df['완료실적'] = pd.to_datetime(df['완료실적'], errors='coerce')
    if work == 'H01':
        df_per_day_1 = df[['정반_코드', '취부팀_코드', '착수실적', '완료실적', '실적공기', 'H01']]
        if factory:
            df_per_day_1 = df_per_day_1[df_per_day_1['정반_코드'] == factory]
        else:
            df_per_day_1 = df[['취부팀_코드', '착수실적', '완료실적', '실적공기', 'H01']]

        idx_1 = df_per_day_1[df_per_day_1['취부팀_코드'] == 'XXX'].index
        df_per_day_1 = df_per_day_1.drop(idx_1)
        df_per_day_1 = df_per_day_1.dropna(axis=0)

        df_per_day_1['하루_취부'] = df_per_day_1['H01'] / df_per_day_1['실적공기']

        df_per_day_1_ex = df_per_day_1[df_per_day_1['취부팀_코드'] == team_name]
        df_team_name = df_per_day_1_ex

    elif work == 'H02':
        df_per_day_2 = df[['정반_코드', '용접팀_코드', '착수실적', '완료실적', '실적공기', 'H02']]
        if factory:
            df_per_day_2 = df_per_day_2[df_per_day_2['정반_코드'] == factory]

        idx_2 = df_per_day_2[df_per_day_2['용접팀_코드'] == 'XXX'].index
        df_per_day_2 = df_per_day_2.drop(idx_2)
        df_per_day_2 = df_per_day_2.dropna(axis=0)

        df_per_day_2['하루_용접'] = df_per_day_2['H02'] / df_per_day_2['실적공기']

        df_per_day_2_ex = df_per_day_2[df_per_day_2['용접팀_코드'] == team_name]
        df_team_name = df_per_day_2_ex

    return df_team_name

In [15]:
df_example = generating_df_team_name('H02', 'B05', 'M031')
df_example = df_example.sort_values('착수실적')
df_example.index = range(len(df_example))

df_example

,정반_코드,용접팀_코드,착수실적,완료실적,실적공기,H02,하루_용접
0,M031,B05,2022-12-02,2022-12-16,11.0,414,37.636364
1,M031,B05,2022-12-03,2022-12-19,12.0,520,43.333333
2,M031,B05,2022-12-12,2022-12-26,10.0,77,7.700000
3,M031,B05,2022-12-12,2022-12-26,10.0,77,7.700000
4,M031,B05,2022-12-15,2022-12-23,6.0,46,7.666667
...,...,...,...,...,...,...,...
246,M031,B05,2025-04-05,2025-04-15,8.0,1156,144.500000
247,M031,B05,2025-04-07,2025-04-17,9.0,1042,115.777778
248,M031,B05,2025-04-08,2025-04-21,10.0,1577,157.700000
249,M031,B05,2025-04-11,2025-04-25,11.0,1331,121.000000


In [24]:
df_example['용접팀_코드'] = 'H31'
df_example

,정반_코드,용접팀_코드,착수실적,완료실적,실적공기,H02,하루_용접
0,M031,H31,2022-12-02,2022-12-16,11.0,414,37.636364
1,M031,H31,2022-12-03,2022-12-19,12.0,520,43.333333
2,M031,H31,2022-12-12,2022-12-26,10.0,77,7.700000
3,M031,H31,2022-12-12,2022-12-26,10.0,77,7.700000
4,M031,H31,2022-12-15,2022-12-23,6.0,46,7.666667
...,...,...,...,...,...,...,...
246,M031,H31,2025-04-05,2025-04-15,8.0,1156,144.500000
247,M031,H31,2025-04-07,2025-04-17,9.0,1042,115.777778
248,M031,H31,2025-04-08,2025-04-21,10.0,1577,157.700000
249,M031,H31,2025-04-11,2025-04-25,11.0,1331,121.000000


In [19]:
datetime.strftime(df_example.loc[0, '착수실적'], '%Y%m%d')

'20221202'

In [22]:
df_capacity = pd.read_excel('../data/capacity data.xlsx', sheet_name='B05')
ser_date = pd.Series(pd.date_range(str(df_capacity.loc[0, '날짜']), str(df_example.loc[df_example.shape[0] - 1, '완료실적'])))

In [23]:
ser_date

0     2022-12-02
1     2022-12-03
2     2022-12-04
3     2022-12-05
4     2022-12-06
         ...    
871   2025-04-21
872   2025-04-22
873   2025-04-23
874   2025-04-24
875   2025-04-25
Length: 876, dtype: datetime64[ns]

In [38]:
def make_capacity(work, team_name, factory=None):
    df = generating_df_team_name(work, team_name, factory)
    df = df.sort_values('착수실적')
    df.index = range(len(df))


    df_capacity = pd.read_excel('../data/capacity data.xlsx', sheet_name=team_name)

    ser_date = pd.Series(pd.date_range(str(df_capacity.loc[0, '날짜']), str(df_capacity.loc[df_capacity.shape[0] - 1, '날짜'])))
    df_capacity['날짜'] = ser_date

    if factory:
        df_capacity['정반_코드'] = factory

    # 누적 노동량 컬럼 초기화
    df_capacity['Capacity'] = 0.0

    # 날짜 순회
    for i in range(df_capacity.shape[0]):
        today = df_capacity.loc[i, '날짜'].date()

        # 이전 날짜의 Capacity 가져오기
        if i > 0:
            df_capacity.loc[i, 'Capacity'] = df_capacity.loc[i - 1, 'Capacity']

        # 각 작업 순회
        for j in range(df.shape[0]):
            start_date = df.loc[j, '착수실적'].date()
            end_date = df.loc[j, '완료실적'].date()
            if work == 'H01':
                daily_amount = df.loc[j, '하루_취부']
            elif work == 'H02':
                daily_amount = df.loc[j, '하루_용접']

            # 착수일이면 증가
            if today == start_date:
                df_capacity.loc[i, 'Capacity'] += daily_amount

            # 완료일이면 감소 시점 결정
            if today == end_date:
                work_type = df_capacity.loc[i, '근무일 여부']

                if work_type == '근무일':
                    df_capacity.loc[i, 'Capacity'] -= daily_amount
                elif work_type == '휴일':
                    # 다음 근무일 찾아서 차감
                    for k in range(i + 1, df_capacity.shape[0]):
                        if df_capacity.loc[k, '근무일 여부'] == '근무일':
                            df_capacity.loc[k, 'Capacity'] -= daily_amount
                            break  # 한 번만 차감해야 하니까 break

    return df_capacity

In [30]:
df_example = make_capacity('H02', 'B05', 'M031')
df_example.to_excel('../capacity_example.xlsx')

In [39]:
make_capacity('H02', 'B05', 'M031')

,날짜,근무일 여부,정반_코드,Capacity
0,2022-12-02,근무일,M031,37.636364
1,2022-12-03,휴일,M031,80.969697
2,2022-12-04,휴일,M031,80.969697
3,2022-12-05,근무일,M031,80.969697
4,2022-12-06,근무일,M031,80.969697
...,...,...,...,...
875,2025-04-25,근무일,M031,322.689394
876,2025-04-26,휴일,M031,322.689394
877,2025-04-27,휴일,M031,322.689394
878,2025-04-28,근무일,M031,322.689394


In [41]:
generating_df_team_name('H02', 'B02', 'M211')

,정반_코드,용접팀_코드,착수실적,완료실적,실적공기,H02,하루_용접
4,M211,B02,2022-11-23,2022-12-13,15.0,386,25.733333
40,M211,B02,2022-12-09,2022-12-15,5.0,326,65.200000
101,M211,B02,2022-11-28,2022-12-22,19.0,666,35.052632
105,M211,B02,2022-12-09,2022-12-19,7.0,250,35.714286
129,M211,B02,2022-12-09,2022-12-22,10.0,540,54.000000
...,...,...,...,...,...,...,...
11738,M211,B02,2025-04-07,2025-04-16,8.0,925,115.625000
11755,M211,B02,2025-04-07,2025-04-18,10.0,595,59.500000
11793,M211,B02,2025-04-10,2025-04-21,8.0,1761,220.125000
11824,M211,B02,2025-04-14,2025-04-23,8.0,0,0.000000
